### Extracción de Datos

In [854]:
# Importar librerías
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

In [855]:
# Extraer datos de tablas
products_df = pd.read_csv('20230223_productos.csv')
sales_df = pd.read_csv('ventas.csv')

### Limpieza de Datos

#### Productos

In [856]:
# Definir columnas relevantes de la tabla de productos
relevant_columns = [
        "Material", "Productos_Por_Empaque", "MLSize", "Returnability", 
        "GlobalFlavor", "Container", "GlobalCategory", "BrandGrouper", 
        "Presentation", "Brand"
    ]

# Extraer solo columnas relevantes de la tabla de productos
products_df = products_df[relevant_columns]

products_df.head()

,Material,Productos_Por_Empaque,MLSize,Returnability,GlobalFlavor,Container,GlobalCategory,BrandGrouper,Presentation,Brand
0,9465,6,600,NO RETORNABLE,LIMÓN,PLASTICO,AGUA,CIEL,600 ML NR,CIEL EXPRIM
1,14450,6,500,NO RETORNABLE,ARÁNDANO ACAÍ,PLASTICO,BEBIDAS EMERGENTES,POWERADE,500 ML NR PET,POWERADE FIT
2,148,24,500,RETORNABLE,UVA,VIDRIO,REFRESCOS,FANTA,500 ML RET,FANTA
3,1088,4,3800,NO RETORNABLE,SIN SABOR,PLASTICO,AGUA,SIERRAZUL,GALÓN 3.8 LTS.,SIERRAZUL AGUA PURIFICADA
4,2142,6,340,NO RETORNABLE,SIN SABOR,VIDRIO,AGUA,TOPO CHICO,12 OZ. NR VIDRIO,TOPO CHICO A.M.


### Transformación de Datos

#### Productos

##### Extraer gramos

In [857]:
# Función para convertir columna de presentación a indicador de gramos
def convert_presentation_to_grams(row):
    if row['MLSize'] == 0:
        if 'KG' in row['Presentation']:
            return int(float(row['Presentation'].split()[1]) * 1000)
        elif 'GR' in row['Presentation']:
            return int(row['Presentation'].split()[1])
    return 0

In [858]:
# Convertir toda la columna de presentación a indicador de gramos en tabla de productos
products_df['Presentation'] = products_df.apply(convert_presentation_to_grams, axis=1)

products_df.head()

,Material,Productos_Por_Empaque,MLSize,Returnability,GlobalFlavor,Container,GlobalCategory,BrandGrouper,Presentation,Brand
0,9465,6,600,NO RETORNABLE,LIMÓN,PLASTICO,AGUA,CIEL,0,CIEL EXPRIM
1,14450,6,500,NO RETORNABLE,ARÁNDANO ACAÍ,PLASTICO,BEBIDAS EMERGENTES,POWERADE,0,POWERADE FIT
2,148,24,500,RETORNABLE,UVA,VIDRIO,REFRESCOS,FANTA,0,FANTA
3,1088,4,3800,NO RETORNABLE,SIN SABOR,PLASTICO,AGUA,SIERRAZUL,0,SIERRAZUL AGUA PURIFICADA
4,2142,6,340,NO RETORNABLE,SIN SABOR,VIDRIO,AGUA,TOPO CHICO,0,TOPO CHICO A.M.


##### One-Hot a Categóricas

In [859]:
# Definir columnas categóricas a codificar
categorical_columns = ["Returnability", "GlobalFlavor", "Container", "GlobalCategory", "BrandGrouper", "Brand"]

# Aplicar One-Hot Encoding a las columnas categóricas
onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
categorical_encoded = onehot_encoder.fit_transform(products_df[categorical_columns])

# Convertir los datos codificados a un DataFrame
categorical_encoded_df = pd.DataFrame(categorical_encoded)

# Eliminar las columnas categóricas originales de products_df y concatenar con las columnas codificadas
products_df = pd.concat([products_df.drop(columns=categorical_columns).reset_index(drop=True), categorical_encoded_df], axis=1)

products_df.head()

,Material,Productos_Por_Empaque,MLSize,Presentation,0,1,2,3,4,5,...,154,155,156,157,158,159,160,161,162,163
0,9465,6,600,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14450,6,500,0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,148,24,500,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1088,4,3800,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2142,6,340,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


##### Normalizar

In [860]:
# Escalar todas las columnas (numéricas y de One-Hot Encoding) para que estén en la misma escala
products_df.columns = products_df.columns.astype(str)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(products_df.drop(columns=['Material']))

##### Componentes Principales

In [861]:
# Aplicar PCA con 50 componentes
pca = PCA(n_components=50)
pca_data = pca.fit_transform(scaled_data)

# Calcular la varianza acumulada para cada número de componentes
cumulative_variance = np.cumsum(pca.explained_variance_ratio_) * 100

# Mostrar la variabilidad explicada
print("Varianza acumulada con 50 componentes: {:.2f}%".format(cumulative_variance[-1]))

Varianza acumulada con 50 componentes: 94.45%


In [862]:
# Crear un DataFrame final con el 'Material' como id y los componentes principales
products_pca_df = pd.DataFrame(pca_data)
products_pca_df.insert(0, 'product_id', products_df['Material'].values)
products_pca_df.head()

,product_id,0,1,2,3,4,5,6,7,8,...,40,41,42,43,44,45,46,47,48,49
0,9465,0.211190,-0.584249,-0.514844,0.245640,0.461752,0.141713,-0.346612,0.040673,0.411813,...,0.052300,-0.020317,-0.268344,-0.027920,0.085601,0.127787,0.066068,0.018365,-0.049061,-0.001235
1,14450,0.686275,-0.398923,-0.578620,-0.190855,-0.191997,0.046877,-0.230070,0.119015,-0.057407,...,0.078341,-0.032772,-0.011288,0.016023,0.066997,-0.004025,-0.037804,-0.060537,0.061597,0.071341
2,148,-0.508888,0.009802,1.388761,0.327996,-0.536266,-0.722467,-0.351427,0.554520,0.474337,...,-0.057720,-0.001852,0.095436,-0.243673,-0.095382,-0.044547,0.256238,-0.109287,-0.112790,-0.000802
3,1088,0.206965,-0.556361,-0.438046,0.322961,0.597207,-0.001359,-0.193804,0.021644,0.279172,...,0.056012,-0.099183,0.079751,-0.193726,-0.210050,0.166551,0.199524,0.389754,-0.058843,0.002765
4,2142,0.449714,0.232054,0.481005,0.343943,0.830362,-0.025471,-0.433537,0.424208,0.789216,...,-0.111001,0.024841,0.090539,0.060911,-0.134664,-0.096036,-0.034007,-0.041045,-0.003662,0.024120


##### Matriz de Características

In [863]:
# Definir matriz, excluir columnas no relacionadas a las caracteristicas
feature_matrix = products_pca_df.drop(columns=['product_id']).values

#### Ventas

##### Convertir Calmonth a DateTime

In [864]:
sales_df['calmonth'] = pd.to_datetime(sales_df['calmonth'], format='%Y%m').dt.to_period('M')

sales_df.head()

,CustomerId,material,calmonth,uni_box
0,499920078,9151,2019-09,0.4364
1,499920078,2287,2019-09,3.1701
2,499920078,4526,2019-09,0.2818
3,499920078,14050,2019-09,0.2642
4,499920078,1333,2019-09,2.1134


##### Renombrar Columnas

In [865]:
sales_df = sales_df.rename(columns={
    'CustomerId': 'customer_id',
    'material': 'product_id',
    'calmonth': 'sale_date',
    'uni_box': 'sale_gallons'
})

sales_df.head()

,customer_id,product_id,sale_date,sale_gallons
0,499920078,9151,2019-09,0.4364
1,499920078,2287,2019-09,3.1701
2,499920078,4526,2019-09,0.2818
3,499920078,14050,2019-09,0.2642
4,499920078,1333,2019-09,2.1134


##### Agrupar por Mes

In [866]:
sales_df = sales_df.groupby(['customer_id', 'product_id', 'sale_date'], as_index=False)['sale_gallons'].sum()

sales_df.head()

,customer_id,product_id,sale_date,sale_gallons
0,499920078,1,2020-02,1.5005
1,499920078,24,2019-09,0.5284
2,499920078,24,2019-11,0.5284
3,499920078,24,2020-01,0.5284
4,499920078,24,2020-04,0.5284


##### Transponer Meses

In [867]:
# Crear una tabla dinámica para reorganizar los datos
pivot_df = sales_df.pivot_table(
    index=['customer_id', 'product_id'],  # Las combinaciones de cliente y producto como índice
    columns='sale_date',                   # Las fechas como columnas (enero, febrero, etc.)
    values='sale_gallons',                 # Los valores de venta
    aggfunc='sum',                         # Sumar las ventas en caso de múltiples registros
    fill_value=0                           # Rellenar con 0 en caso de que no haya ventas para un mes
)

# Restablecer el índice para tener un DataFrame limpio
sales_df = pivot_df.reset_index()

sales_df.head()

sale_date,customer_id,product_id,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
0,499920078,1,0.0000,0.0000,0.0000,0.0000,0.0000,1.5005,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000
1,499920078,24,0.5284,0.0000,0.5284,0.0000,0.5284,0.0000,0.0000,0.5284,...,0.5284,0.0000,0.000,0.0000,0.5284,0.000,0.5284,0.0000,0.5284,0.5284
2,499920078,100,16.9072,8.4536,19.0206,16.9072,16.9072,14.7938,14.7938,25.3608,...,8.4536,8.4536,10.567,12.6804,8.4536,10.567,6.3402,8.4536,10.5670,6.3402
3,499920078,101,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000
4,499920078,117,2.1134,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000


##### Calcular Frecuencia de Ventas por Producto

In [868]:
# Calculamos el número de meses con ventas para cada producto entre el total de meses
sale_frequency = (sales_df.iloc[:, 2:] > 0).sum(axis=1) / (sales_df.iloc[:, 2:]).shape[1]
sales_df.insert(2, "sale_frequency", sale_frequency)

sales_df.head()

sale_date,customer_id,product_id,sale_frequency,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
0,499920078,1,0.025,0.0000,0.0000,0.0000,0.0000,0.0000,1.5005,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000
1,499920078,24,0.475,0.5284,0.0000,0.5284,0.0000,0.5284,0.0000,0.0000,...,0.5284,0.0000,0.000,0.0000,0.5284,0.000,0.5284,0.0000,0.5284,0.5284
2,499920078,100,1.000,16.9072,8.4536,19.0206,16.9072,16.9072,14.7938,14.7938,...,8.4536,8.4536,10.567,12.6804,8.4536,10.567,6.3402,8.4536,10.5670,6.3402
3,499920078,101,0.075,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000
4,499920078,117,0.025,2.1134,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000


##### Calcular Magnitud de Ventas por Producto

In [869]:
# Calculamos el total de galones vendidos por cada producto
sales_df['sales_per_product'] = sales_df.iloc[:, 3:].sum(axis=1)

# Calculamos la proporción de galones vendidos por producto
sales_df.insert(3, "sale_proportion", sales_df['sales_per_product'] / sales_df.groupby('customer_id')['sales_per_product'].transform('sum'))

# Eliminamos la columna temporal 'sales_per_product'
sales_df.drop(columns=['sales_per_product'], inplace=True)

sales_df.head()

sale_date,customer_id,product_id,sale_frequency,sale_proportion,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,...,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12
0,499920078,1,0.025,0.000243,0.0000,0.0000,0.0000,0.0000,0.0000,1.5005,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000
1,499920078,24,0.475,0.001629,0.5284,0.0000,0.5284,0.0000,0.5284,0.0000,...,0.5284,0.0000,0.000,0.0000,0.5284,0.000,0.5284,0.0000,0.5284,0.5284
2,499920078,100,1.000,0.121360,16.9072,8.4536,19.0206,16.9072,16.9072,14.7938,...,8.4536,8.4536,10.567,12.6804,8.4536,10.567,6.3402,8.4536,10.5670,6.3402
3,499920078,101,0.075,0.001028,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000
4,499920078,117,0.025,0.000343,2.1134,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000


##### Limpiar Tabla

In [870]:
sales_df = sales_df[['customer_id', 'product_id', 'sale_frequency', 'sale_proportion']]

sales_df.head()

sale_date,customer_id,product_id,sale_frequency,sale_proportion
0,499920078,1,0.025,0.000243
1,499920078,24,0.475,0.001629
2,499920078,100,1.000,0.121360
3,499920078,101,0.075,0.001028
4,499920078,117,0.025,0.000343


### Modelación

#### Producto Nuevo

##### Definir Producto Nuevo

In [871]:
new_product_data = {
    'Material_desc': 'SPRITE 600 ML NR 24B', 
    'Productos_Por_Empaque': 24, 
    'BrandPresRet': 'SPRITE 600 ML NO RETORNABLE', 
    'ProdKey': 'SABORES INDIVIDUAL NO RETORNABLE', 
    'Brand': 'SPRITE', 
    'Presentation': '600 ML NR', 
    'MLSize': 600, 
    'Returnability': 'NO RETORNABLE', 
    'Pack': '600 ML', 
    'Size': 'INDIVIDUAL', 
    'Flavor': 'LIMA LIMON', 
    'Container': 'PLASTICO', 
    'Ncb': 0, 
    'ProductType': 'SABORES REGULAR', 
    'ProductCategory': 'REFRESCOS', 
    'SegAg': 'SABORES', 
    'SegDet': 'SABORES REGULAR', 
    'GlobalCategory': 'REFRESCOS', 
    'GlobalSubcategory': 'SABORES', 
    'BrandGrouper': 'SPRITE', 
    'GlobalFlavor': 'LIMA LIMÓN'
    }

new_product = pd.DataFrame([new_product_data])

new_product.head()

,Material_desc,Productos_Por_Empaque,BrandPresRet,ProdKey,Brand,Presentation,MLSize,Returnability,Pack,Size,...,Container,Ncb,ProductType,ProductCategory,SegAg,SegDet,GlobalCategory,GlobalSubcategory,BrandGrouper,GlobalFlavor
0,SPRITE 600 ML NR 24B,24,SPRITE 600 ML NO RETORNABLE,SABORES INDIVIDUAL NO RETORNABLE,SPRITE,600 ML NR,600,NO RETORNABLE,600 ML,INDIVIDUAL,...,PLASTICO,0,SABORES REGULAR,REFRESCOS,SABORES,SABORES REGULAR,REFRESCOS,SABORES,SPRITE,LIMA LIMÓN


##### Transformar Producto Nuevo a Formato de Tabla de Productos

In [872]:
# Extraer columnas relevantes
new_product = new_product[relevant_columns[1:]]

# Extraer gramos
new_product['Presentation'] = new_product.apply(convert_presentation_to_grams, axis=1)

# One-Hot
categorical_encoded = onehot_encoder.transform(new_product[categorical_columns])
categorical_encoded_df = pd.DataFrame(categorical_encoded)
new_product = pd.concat([new_product.drop(columns=categorical_columns).reset_index(drop=True), categorical_encoded_df], axis=1)

# Normalizar
new_product.columns = new_product.columns.astype(str)
scaled_data = scaler.transform(new_product)

# Componentes Principales
pca_data = pca.transform(scaled_data)
new_product = pd.DataFrame(pca_data)

new_product.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.519878,-0.538331,0.003293,0.012827,0.03226,0.671044,0.680304,1.109261,-0.551382,-0.552979,...,0.013902,0.016777,-0.006051,-0.02626,0.02353,-0.012016,0.002626,0.085541,-0.003569,-0.151464


#### Similitud Coseno

In [873]:
# Calcular similitud coseno entre el producto nuevo y todos los productos
similarities = cosine_similarity(new_product, feature_matrix)[0, :]

# Juntar product_id con su similitud
products_pca_df['product_similarity'] = similarities

products_similarities = products_pca_df[['product_id', 'product_similarity']]

products_similarities.head()

,product_id,product_similarity
0,9465,-0.002100
1,14450,-0.092899
2,148,-0.045351
3,1088,-0.023111
4,2142,-0.146547


##### Unir Tablas

In [874]:
merged_df = pd.merge(sales_df, products_similarities, on='product_id', how='left')

merged_df.head()

,customer_id,product_id,sale_frequency,sale_proportion,product_similarity
0,499920078,1,0.025,0.000243,-0.095636
1,499920078,24,0.475,0.001629,0.988947
2,499920078,100,1.000,0.121360,-0.096204
3,499920078,101,0.075,0.001028,0.671834
4,499920078,117,0.025,0.000343,-0.049841


#### Producto de Indicadores

In [875]:
# Calcula el producto de los indicadores
merged_df['compatibility'] = merged_df['sale_frequency'] * merged_df['sale_proportion'] * merged_df['product_similarity']

merged_df.head()

,customer_id,product_id,sale_frequency,sale_proportion,product_similarity,compatibility
0,499920078,1,0.025,0.000243,-0.095636,-5.819561e-07
1,499920078,24,0.475,0.001629,0.988947,7.650236e-04
2,499920078,100,1.000,0.121360,-0.096204,-1.167531e-02
3,499920078,101,0.075,0.001028,0.671834,5.182246e-05
4,499920078,117,0.025,0.000343,-0.049841,-4.271702e-07


In [877]:
merged_df = merged_df.drop(columns=['sale_frequency', 'sale_proportion', 'product_similarity'])
merged_df.head()

,customer_id,product_id,compatibility
0,499920078,1,-5.819561e-07
1,499920078,24,7.650236e-04
2,499920078,100,-1.167531e-02
3,499920078,101,5.182246e-05
4,499920078,117,-4.271702e-07


#### Ordenar por Compatibilidad

In [ ]:
# Ordena el dataframe por 'compatibility' y 'customer_id' en orden descendente
sorted_df = merged_df.sort_values(by=['compatibility', 'customer_id'], ascending=[False, True])

sorted_df.head()

,customer_id,product_id,compatibility
134338,500399389,304,0.105772
80951,500180311,375,0.087314
180672,510275595,197,0.069417
209512,510543541,408,0.060896
184121,510301968,375,0.059800


#### Gestionar Clientes Duplicados 

In [880]:
### Aquí definimos el criterio de filtrado por ejemplo si queremos juntar los mejores 10 productos de cada cliente

# Por ahora solo dejamos el mejor producto por cliente
unique_df = sorted_df.drop_duplicates(subset='customer_id', keep='first')

unique_df.head()

,customer_id,product_id,compatibility
134338,500399389,304,0.105772
80951,500180311,375,0.087314
180672,510275595,197,0.069417
209512,510543541,408,0.060896
184121,510301968,375,0.059800
